In [1]:
import bokeh.io as io
import bokeh.models as models
import bokeh.plotting as plotting
import bokeh.resources as resources
import pandas as pd
from bokeh.models import ColumnDataSource
import math
from bokeh.io import output_notebook
import bokeh.palettes
import pickle
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, dodge
import math
from bokeh.layouts import column, row, grid, gridplot

In [2]:
!set PYTHONPATH 

In [3]:
import viewer.app.data as data

In [4]:
!export PYTHONPATH=/Users/nehamuramalla/pythonrobotics/scouting2022/scouter:/Users/nehamuramalla/pythonrobotics/scouting2022:/Users/nehamuramalla/scouting2020

In [5]:
from bokeh.io import output_notebook

In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
import pickle
import pandas as pd

In [8]:
measures = data.get_data()[0]
matches = data.get_data()[1]

Reading Viewer Data from /Users/nehamuramalla/pythonrobotics/scouting2022/viewer/app/viewer_data.json


In [9]:
measures.head(4)

,match,measure1,measure2,measure_type,phase,task,team_number
0,qm1,left,-1,categorical,auto,start_pos,frc1294
1,qm1,1,-1,count,auto,pickup_field,frc1294
10,qm1,2,-1,count,tele,pickup-field,frc4512
100,qm1,0,-1,count,endgame,penalty_count,frc4915


In [11]:
m = measures[measures.team_number == 'frc1318']
m['phase_task'] = m['phase'] + "_" + m['task']
m = m[['match', 'team_number', 'phase_task', 'measure1']]
filter1 = m['phase_task'].isin(['auto_upper', 'auto_lower', 'tele_upper', 'tele_lower', 'endgame_hangar_level_end'])
m = m[filter1]
m.head()

<ipython-input-11-bec3fa7095ba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['phase_task'] = m['phase'] + "_" + m['task']


,match,team_number,phase_task,measure1
1275,qm11,frc1318,auto_upper,0
1282,qm11,frc1318,tele_upper,3
1291,qm11,frc1318,tele_lower,1
1338,qm11,frc1318,endgame_hangar_level_end,0
1340,qm11,frc1318,auto_lower,0


In [12]:
tele_up, tele_low, auto_up, auto_low, hangar = ['tele_up'], ['tele_low'], ['auto_up'], ['auto_low'], ['hangar']
for i in range(0, len(m)):
    if(m.iloc[i]['phase_task'] == 'tele_upper'):
        tele_up.append(m.iloc[i]['measure1'])
    if(m.iloc[i]['phase_task'] == 'tele_lower'):
        tele_low.append(m.iloc[i]['measure1'])
    if(m.iloc[i]['phase_task'] == 'auto_upper'):
        auto_up.append(m.iloc[i]['measure1'])
    if(m.iloc[i]['phase_task'] == 'auto_lower'):
        auto_low.append(m.iloc[i]['measure1'])
    if(m.iloc[i]['phase_task'] == 'endgame_hangar_level_end'):
        hangar.append(m.iloc[i]['measure1'])
# 5 lists with the measure 1 values for each task 

In [13]:
tele_up

['tele_up', '3', '5', '4', '3', '6', '7', '4', '4', '8', '0', '6']

In [14]:
matches

,alliance,match,match_time,station,team_number
0,blue,qm1,2022-03-19 21:15:00,1,frc4915
1,blue,qm1,2022-03-19 21:15:00,2,frc1294
10,red,qm2,2022-03-19 21:25:00,2,frc4579
100,red,qm17,2022-03-19 23:45:00,2,frc2980
101,red,qm17,2022-03-19 23:45:00,3,frc4180
...,...,...,...,...,...
95,red,qm16,2022-03-19 23:37:30,3,frc2929
96,blue,qm17,2022-03-19 23:45:00,1,frc4469
97,blue,qm17,2022-03-19 23:45:00,2,frc4911
98,blue,qm17,2022-03-19 23:45:00,3,frc3786


In [15]:
matches = list(m.match.unique())
matches.insert(0, 'match')
big_boi = [['team', 'frc1318'], matches, tele_up, tele_low, auto_up, auto_low, hangar]
big_boi

#autofill empty values with 0
#make the match list 1D

[['team', 'frc1318'],
 ['match',
  'qm11',
  'qm16',
  'qm20',
  'qm24',
  'qm27',
  'qm33',
  'qm38',
  'qm5',
  'qm50',
  'qm54',
  'qm58'],
 ['tele_up', '3', '5', '4', '3', '6', '7', '4', '4', '8', '0', '6'],
 ['tele_low', '1', '0', '1', '0', '0', '1', '1', '0', '2', '0', '0'],
 ['auto_up', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1'],
 ['auto_low', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['hangar', '0', '0', '2', '0', '0', '0', '0', '2', '2', '0', '0']]

In [16]:
for l in range(1, len(big_boi[1])):
    for i in big_boi:
        if(i != big_boi[0]):
            if (len(big_boi[1]) - len(i)) > 0:
                i.append('0')
        if(i == big_boi[0]):
            if (len(big_boi[1]) - len(i)) > 0:
                i.append('frc1318')
big_boi

[['team',
  'frc1318',
  'frc1318',
  'frc1318',
  'frc1318',
  'frc1318',
  'frc1318',
  'frc1318',
  'frc1318',
  'frc1318',
  'frc1318',
  'frc1318'],
 ['match',
  'qm11',
  'qm16',
  'qm20',
  'qm24',
  'qm27',
  'qm33',
  'qm38',
  'qm5',
  'qm50',
  'qm54',
  'qm58'],
 ['tele_up', '3', '5', '4', '3', '6', '7', '4', '4', '8', '0', '6'],
 ['tele_low', '1', '0', '1', '0', '0', '1', '1', '0', '2', '0', '0'],
 ['auto_up', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1'],
 ['auto_low', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['hangar', '0', '0', '2', '0', '0', '0', '0', '2', '2', '0', '0']]

In [17]:
dictchart = {'team': big_boi[0][1:], 'match': big_boi[1][1:], 'tele_up': big_boi[2][1:], 'tele_low': big_boi[3][1:], 'auto_up': big_boi[4][1:], 'auto_low':big_boi[5][1:], 'hangar':big_boi[6][1:]}

In [18]:
#dictchart

In [19]:
import bokeh.io as io
import bokeh.models as models
import bokeh.plotting as plotting
import bokeh.resources as resources
import pandas as pd
from bokeh.models import ColumnDataSource
import math
from bokeh.io import output_notebook
import bokeh.palettes
import pickle
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, dodge
import math
from bokeh.layouts import column, row, grid, gridplot

In [20]:
df = pd.DataFrame(dictchart)
df.drop(columns = 'team')

,match,tele_up,tele_low,auto_up,auto_low,hangar
0,qm11,3,1,0,0,0
1,qm16,5,0,0,0,0
2,qm20,4,1,0,0,2
3,qm24,3,0,1,0,0
4,qm27,6,0,0,0,0
5,qm33,7,1,0,0,0
6,qm38,4,1,1,0,0
7,qm5,4,0,0,0,2
8,qm50,8,2,1,0,2
9,qm54,0,0,0,0,0


In [21]:
df['tele_up'] = df['tele_up'].astype('int32')
df['tele_low'] = df['tele_low'].astype('int32')
df['auto_up'] = df['auto_up'].astype('int32')
df['auto_low'] = df['auto_low'].astype('int32')
df['hangar'] = df['hangar'].astype('int32')

In [22]:
bob = ['match', 'team']
for l in bob:
    for i in range(0, len(df[l])):
        if(l == 'match'):
            df[l].loc[i] = int(df[l].iloc[i][2:])
        if(l == 'team'):
            df[l].loc[i] = int(df[l].iloc[i][3:])
df.sort_values(by=['match'], inplace = True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
df

,team,match,tele_up,tele_low,auto_up,auto_low,hangar
7,1318,5,4,0,0,0,2
0,1318,11,3,1,0,0,0
1,1318,16,5,0,0,0,0
2,1318,20,4,1,0,0,2
3,1318,24,3,0,1,0,0
4,1318,27,6,0,0,0,0
5,1318,33,7,1,0,0,0
6,1318,38,4,1,1,0,0
8,1318,50,8,2,1,0,2
9,1318,54,0,0,0,0,0


In [24]:
df['match'] = df['match'].astype(str)

In [25]:
type(df['match'].iloc[0])

str

In [26]:
source = df

p = figure(x_range = df['match'],
            y_range = (0,15), 
           title = "IRS 1318 Team Chart", 
           plot_width = 600, 
           height = 400, 
           toolbar_location = None, 
           tools="hover", tooltips="@team @match, $name: @$name")

p.vbar_stack(['auto_low', 'auto_up', 'tele_low', 'tele_up','hangar'],
                             x = 'match', 
                             color = bokeh.palettes.Paired[5][:], 
                             width = 0.9,
                             source = source,
                             legend_label = ['auto_lower', 'auto_upper', 'tele_lower', 'tele_upper','hangar'])
                            

# display the plot
plotting.show(p)

In [27]:
df['team'] = "1318"

In [28]:
df['match'] = matches

ValueError: Length of values (12) does not match length of index (11)

In [ ]:
df['tele_up'] = tele_up

In [29]:
df['tele_low'] = tele_low

ValueError: Length of values (12) does not match length of index (11)

In [30]:
df['auto_up'] = auto_up

ValueError: Length of values (12) does not match length of index (11)

In [31]:
tele_up

['tele_up', '3', '5', '4', '3', '6', '7', '4', '4', '8', '0', '6']

In [32]:
df['auto_low'] = auto_low
df['hangar'] = hangar
df.drop(columns = ['team'])

ValueError: Length of values (12) does not match length of index (11)

ValueError: Length of values (12) does not match length of index (11)

,match,tele_up,tele_low,auto_up,auto_low,hangar
7,5,4,0,0,0,2
0,11,3,1,0,0,0
1,16,5,0,0,0,0
2,20,4,1,0,0,2
3,24,3,0,1,0,0
4,27,6,0,0,0,0
5,33,7,1,0,0,0
6,38,4,1,1,0,0
8,50,8,2,1,0,2
9,54,0,0,0,0,0


In [35]:
measures

,match,measure1,measure2,measure_type,phase,task,team_number
0,qm1,left,-1,categorical,auto,start_pos,frc1294
1,qm1,1,-1,count,auto,pickup_field,frc1294
10,qm1,2,-1,count,tele,pickup-field,frc4512
100,qm1,0,-1,count,endgame,penalty_count,frc4915
1000,qm8,0,-1,count,endgame,penalty_count,frc949
...,...,...,...,...,...,...,...
995,qm8,0,0,count,auto,hp,frc949
996,qm8,0,-1,count,endgame,hangar_level_end,frc949
997,qm8,false,-1,boolean,endgame,fall_over,frc949
998,qm8,0,-1,count,endgame,hangar_level_start,frc949


In [ ]:
is 